# San Francisco Street Trees!
Some fun exploration of the data on SF street trees on BigQuery.

In [0]:
from google.colab import auth
auth.authenticate_user() 
project_id = "cellular-scion-217900"

In [0]:
import pandas as pd
import altair as alt

## Top Tree Care Takers

To start, let's explore who's caring for the trees...

In [0]:
%%bigquery --project $project_id sfTreeCareTakers
SELECT
  care_taker,
  COUNT(*) AS trees
FROM
  `bigquery-public-data.san_francisco_trees.street_trees`
WHERE
  plant_type = "Tree"
GROUP BY
  care_taker
ORDER BY
  trees DESC
LIMIT
  15

,care_taker,trees
0,Private,160657
1,DPW,26644
2,SFUSD,1061
3,Port,723
4,Rec/Park,721
5,PUC,261
6,DPW for City Agency,199
7,MTA,104
8,Dept of Real Estate,87
9,Purchasing Dept,85


In [0]:
alt.Chart(sfTreeCareTakers).mark_bar().encode(
    y=alt.Y("care_taker:O", sort=alt.EncodingSortField(field="trees", op="count", order='ascending'), axis = alt.Axis(title="Care Takers")),
    x=alt.X("trees:Q", axis = alt.Axis(title="Number of Trees"), scale=alt.Scale(type='log'))
)

Chart({
  data:                  care_taker   trees
  0                   Private  160657
  1                       DPW   26644
  2                     SFUSD    1061
  3                      Port     723
  4                  Rec/Park     721
  5                       PUC     261
  6       DPW for City Agency     199
  7                       MTA     104
  8       Dept of Real Estate      87
  9           Purchasing Dept      85
  10        Housing Authority      73
  11                Fire Dept      69
  12              Health Dept      54
  13              Police Dept      50
  14  Mayor Office of Housing      36,
  encoding: EncodingWithFacet({
    x: X({
      axis: Axis({
        title: 'Number of Trees'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'trees:Q'
    }),
    y: Y({
      axis: Axis({
        title: 'Care Takers'
      }),
      shorthand: 'care_taker:O',
      sort: EncodingSortField({
        field: 'trees',
        op: 'count',
        order: 'ascending'
      })
    })
  }),
  mark: 'bar'
})

Unsurprisingly, the vast majority of trees are cared for by private entities followed by the Department of Public Works.  What does surprise me is the difference between the number of trees cared for by private entities vs. the Department of Public Works.  Perhaps this is just due to ignorance on my part, but I went in thinking that public services would care for quite a few more trees, but I guess I am wrong!  Either way, this is an important query to look at to establish background for the rest of the analysis.

## Most Prominent Tree Types

Next up, lets explore the top tree types in SF!  Afterwards, we can similarly explore where these trees are planted most...

In [0]:
%%bigquery --project $project_id sfTreeTypes
SELECT
  species,
  COUNT(*) AS trees
FROM
  `bigquery-public-data.san_francisco_trees.street_trees`
WHERE
  plant_type = "Tree" AND
  location <> ""
GROUP BY
  species
ORDER BY
  trees DESC
LIMIT
  20

,species,trees
0,Platanus x hispanica :: Sycamore: London Plane,11444
1,Tree(s) ::,10283
2,Metrosideros excelsa :: New Zealand Xmas Tree,8646
3,Lophostemon confertus :: Brisbane Box,8411
4,Pittosporum undulatum :: Victorian Box,7052
5,Tristaniopsis laurina :: Swamp Myrtle,6972
6,Prunus cerasifera :: Cherry Plum,6689
7,Magnolia grandiflora :: Southern Magnolia,6233
8,Ficus microcarpa nitida 'Green Gem' :: Indian ...,5656
9,Arbutus 'Marina' :: Hybrid Strawberry Tree,5558


In [0]:
alt.Chart(sfTreeTypes).mark_bar().encode(
    y=alt.Y("species:O", sort=alt.EncodingSortField(field="trees", op="count", order='ascending'), axis = alt.Axis(title="Tree Species")),
    x=alt.X("trees:Q", axis = alt.Axis(title="Number of Trees")),
    tooltip=['species', 'trees']
)

Chart({
  data:                                               species  trees
  0      Platanus x hispanica :: Sycamore: London Plane  11444
  1                                          Tree(s) ::  10283
  2       Metrosideros excelsa :: New Zealand Xmas Tree   8646
  3               Lophostemon confertus :: Brisbane Box   8411
  4              Pittosporum undulatum :: Victorian Box   7052
  5               Tristaniopsis laurina :: Swamp Myrtle   6972
  6                    Prunus cerasifera :: Cherry Plum   6689
  7           Magnolia grandiflora :: Southern Magnolia   6233
  8   Ficus microcarpa nitida 'Green Gem' :: Indian ...   5656
  9          Arbutus 'Marina' :: Hybrid Strawberry Tree   5558
  10  Prunus serrulata 'Kwanzan' :: Kwanzan Flowerin...   4023
  11             Acacia melanoxylon :: Blackwood Acacia   3924
  12                          Maytenus boaria :: Mayten   3886
  13                        Olea europaea :: Olive Tree   3662
  14            Corymbia ficifolia :: Red Flowering Gum   3529
  15          Callistemon citrinus :: Lemon Bottlebrush   3267
  16                   Ginkgo biloba :: Maidenhair Tree   3176
  17                Pyrus calleryana :: Ornamental Pear   2977
  18              Prunus serrulata :: Ornamental Cherry   2660
  19                    Ulmus parvifolia :: Chinese Elm   2340,
  encoding: EncodingWithFacet({
    tooltip: [Tooltip({
      shorthand: 'species'
    }), Tooltip({
      shorthand: 'trees'
    })],
    x: X({
      axis: Axis({
        title: 'Number of Trees'
      }),
      shorthand: 'trees:Q'
    }),
    y: Y({
      axis: Axis({
        title: 'Tree Species'
      }),
      shorthand: 'species:O',
      sort: EncodingSortField({
        field: 'trees',
        op: 'count',
        order: 'ascending'
      })
    })
  }),
  mark: 'bar'
})

Let's go through some of the most prominant tree types in SF!


- The top tree is the Planatus x hispanica.  This tree also known as the London Plane Tree, and much to its name, it is quite plain indeed.  It is a cross between Platanus occidentalis × Platanus orientalis, and is native to Spain.  You've probably seen quite a few of these without even realizing! [Learn More](https://selectree.calpoly.edu/tree-detail/platanus-%C3%97-hispanica)
![](https://www.ebben.nl/files/treeEbb/images/852x480/platanus-x-hispanica-pyramidalis-high-stem-2.jpg)

- Most trees are trees of unknown variety.  Unfortunately, this isn't the most interesting, so let's move onto the next!
![alt text](https://ak3.picdn.net/shutterstock/videos/12191783/thumb/12.jpg)

- Next up we've got the Metrosideros excelsa.  Also known as the New Zealand Christmas Tree, it has pretty red string-like flowers.  There's a massive one located in Golden Gate Park registered as a California Big Tree, measuring at 66 ft high!  [Learn More](https://selectree.calpoly.edu/tree-detail/metrosideros-excelsa)
![alt text](https://selectree.calpoly.edu/images/0900/39/original/metrosideros-excelsa-tree-flowering.jpg)

- Last in our brief exploration we've got the Lophostemon confertus.  Also known as the Brisbane Box, this tree has got tons of little seed pods that I'm sure you've stepped on at some point. [Learn More](https://selectree.calpoly.edu/tree-detail/lophostemon-confertus)
![alt text](https://keyserver.lucidcentral.org/weeds/data/media/Images/lophostemon_confertus/lophostemonconfertus4fks.jpg)

## Exploration of Districts

San Francisco has many iconic districts, but how do their tree counts stack?  Who has the most trees?

*Note*: In determining which district each tree belongs to, I query'd the sfpd_incidents table and determined the average coordinates for each district, and then found which district each tree was closest to.  Not a perfect method, but good for our purposes!

**For reference: SFPD District Map:**

![alt text](https://hoodwork-production.s3.amazonaws.com/uploads/story/image/10247/sfpdfinal.png)

In [0]:
%%bigquery --project $project_id sfTreeCountsByNeighborhood
SELECT
  B.neighborhood,
  COUNT(*) AS trees
FROM
  `bigquery-public-data.san_francisco_trees.street_trees` AS A,
  (
  SELECT
  tree_id,
  neighborhood
FROM (
  SELECT
    DISTINCT A.tree_id,
    B.neighborhood,
    MIN(ST_DISTANCE(ST_GeogPoint(A.longitude,
          A.latitude),
        ST_GeogPoint(B.longitude,
          B.latitude))) AS distance
  FROM
    `bigquery-public-data.san_francisco_trees.street_trees` AS A,
    (
    SELECT
      pddistrict AS neighborhood,
      AVG(latitude) AS latitude,
      AVG(longitude) AS longitude
    FROM
      `bigquery-public-data.san_francisco_sfpd_incidents.sfpd_incidents`
    GROUP BY
      pddistrict) AS B
  GROUP BY
    A.tree_id,
    B.neighborhood
  ORDER BY
    A.tree_id)
WHERE
  distance IN (
  SELECT
    MIN(distance)
  FROM (
    SELECT
      DISTINCT A.tree_id,
      B.neighborhood,
      MIN(ST_DISTANCE(ST_GeogPoint(A.longitude,
            A.latitude),
          ST_GeogPoint(B.longitude,
            B.latitude))) AS distance
    FROM
      `bigquery-public-data.san_francisco_trees.street_trees` AS A,
      (
      SELECT
        pddistrict AS neighborhood,
        AVG(latitude) AS latitude,
        AVG(longitude) AS longitude
      FROM
        `bigquery-public-data.san_francisco_sfpd_incidents.sfpd_incidents`
      WHERE
        pddistrict <> " "
      GROUP BY
        pddistrict) AS B
    GROUP BY
      A.tree_id,
      B.neighborhood
    ORDER BY
      A.tree_id)
  GROUP BY
    tree_id
  ORDER BY
    tree_id)
GROUP BY
  tree_id,
  neighborhood
ORDER BY
  tree_id) AS B
WHERE
  A.tree_id = B.tree_id
  AND A.plant_type = "Tree"
GROUP BY
  B.neighborhood
ORDER BY 
  trees DESC

,neighborhood,trees
0,MISSION,33071
1,TARAVAL,29149
2,INGLESIDE,27087
3,PARK,22403
4,NORTHERN,19057
5,BAYVIEW,18995
6,RICHMOND,14361
7,SOUTHERN,13173
8,CENTRAL,8404
9,TENDERLOIN,2614


In [0]:
alt.Chart(sfTreeCountsByNeighborhood).mark_bar().encode(
    y=alt.Y("neighborhood:O", sort=alt.EncodingSortField(field="trees", op="count", order='ascending'), axis = alt.Axis(title="Neighborhood")),
    x=alt.X("trees:Q", axis = alt.Axis(title="Number of Trees")),
    tooltip=["neighborhood", 'trees']
)

Chart({
  data:   neighborhood  trees
  0      MISSION  33071
  1      TARAVAL  29149
  2    INGLESIDE  27087
  3         PARK  22403
  4     NORTHERN  19057
  5      BAYVIEW  18995
  6     RICHMOND  14361
  7     SOUTHERN  13173
  8      CENTRAL   8404
  9   TENDERLOIN   2614,
  encoding: EncodingWithFacet({
    tooltip: [Tooltip({
      shorthand: 'neighborhood'
    }), Tooltip({
      shorthand: 'trees'
    })],
    x: X({
      axis: Axis({
        title: 'Number of Trees'
      }),
      shorthand: 'trees:Q'
    }),
    y: Y({
      axis: Axis({
        title: 'Neighborhood'
      }),
      shorthand: 'neighborhood:O',
      sort: EncodingSortField({
        field: 'trees',
        op: 'count',
        order: 'ascending'
      })
    })
  }),
  mark: 'bar'
})

Interestingly, it appears that Mission has the most number of street trees, despite being one of the smaller districts.  This could be due to how a tree's district is determined due to my coordinate calculations, and Mission being in the center being closer to trees than other areas, but it is also an area that holds pride in its high tree count and diversity ([Learn more here!](https://missionlocal.org/2012/06/a-street-guide-to-the-missions-trees/)).  This tree diversity in Mission may be an interesting area to explore, but for now, lets look at how many of the previous top 20 prominent trees are represented in each district.

In [0]:
%%bigquery --project $project_id sfTreeTypesByNeighborhood
SELECT
  A.species,
  B.neighborhood,
  COUNT(*) AS trees
FROM
  `bigquery-public-data.san_francisco_trees.street_trees` AS A,
  (
  SELECT
  tree_id,
  neighborhood
FROM (
  SELECT
    DISTINCT A.tree_id,
    B.neighborhood,
    MIN(ST_DISTANCE(ST_GeogPoint(A.longitude,
          A.latitude),
        ST_GeogPoint(B.longitude,
          B.latitude))) AS distance
  FROM
    `bigquery-public-data.san_francisco_trees.street_trees` AS A,
    (
    SELECT
      pddistrict AS neighborhood,
      AVG(latitude) AS latitude,
      AVG(longitude) AS longitude
    FROM
      `bigquery-public-data.san_francisco_sfpd_incidents.sfpd_incidents`
    GROUP BY
      pddistrict) AS B
  GROUP BY
    A.tree_id,
    B.neighborhood
  ORDER BY
    A.tree_id)
WHERE
  distance IN (
  SELECT
    MIN(distance)
  FROM (
    SELECT
      DISTINCT A.tree_id,
      B.neighborhood,
      MIN(ST_DISTANCE(ST_GeogPoint(A.longitude,
            A.latitude),
          ST_GeogPoint(B.longitude,
            B.latitude))) AS distance
    FROM
      `bigquery-public-data.san_francisco_trees.street_trees` AS A,
      (
      SELECT
        pddistrict AS neighborhood,
        AVG(latitude) AS latitude,
        AVG(longitude) AS longitude
      FROM
        `bigquery-public-data.san_francisco_sfpd_incidents.sfpd_incidents`
      WHERE
        pddistrict <> " "
      GROUP BY
        pddistrict) AS B
    GROUP BY
      A.tree_id,
      B.neighborhood
    ORDER BY
      A.tree_id)
  GROUP BY
    tree_id
  ORDER BY
    tree_id)
GROUP BY
  tree_id,
  neighborhood
ORDER BY
  tree_id) AS B
WHERE
  A.tree_id = B.tree_id
  AND A.plant_type = "Tree"
  AND A.species IN (
  SELECT
    species
  FROM
    `bigquery-public-data.san_francisco_trees.street_trees`
  WHERE
    plant_type = "Tree"
  GROUP BY
    species
  ORDER BY
    COUNT(*) DESC
  LIMIT
    20)
GROUP BY
  A.species,
  B.neighborhood
ORDER BY 
  trees DESC

,species,neighborhood,trees
0,Platanus x hispanica :: Sycamore: London Plane,NORTHERN,2622
1,Metrosideros excelsa :: New Zealand Xmas Tree,TARAVAL,2258
2,Platanus x hispanica :: Sycamore: London Plane,SOUTHERN,2235
3,Tree(s) ::,MISSION,2206
4,Tree(s) ::,PARK,2202
5,Tristaniopsis laurina :: Swamp Myrtle,INGLESIDE,1887
6,Lophostemon confertus :: Brisbane Box,SOUTHERN,1835
7,Arbutus 'Marina' :: Hybrid Strawberry Tree,TARAVAL,1763
8,Platanus x hispanica :: Sycamore: London Plane,CENTRAL,1654
9,Metrosideros excelsa :: New Zealand Xmas Tree,RICHMOND,1629


In [0]:
alt.Chart(sfTreeTypesByNeighborhood).mark_bar().encode(
    y=alt.Y("species:O", sort=alt.EncodingSortField(field="trees", op="count", order='ascending'), axis = alt.Axis(title="Tree Species")),
    x=alt.X("trees:Q", axis = alt.Axis(title="Number of Trees")),
    tooltip=['species', 'trees', "neighborhood"],
    color="neighborhood"
)

Chart({
  data:                                                species neighborhood  trees
  0       Platanus x hispanica :: Sycamore: London Plane     NORTHERN   2622
  1        Metrosideros excelsa :: New Zealand Xmas Tree      TARAVAL   2258
  2       Platanus x hispanica :: Sycamore: London Plane     SOUTHERN   2235
  3                                           Tree(s) ::      MISSION   2206
  4                                           Tree(s) ::         PARK   2202
  5                Tristaniopsis laurina :: Swamp Myrtle    INGLESIDE   1887
  6                Lophostemon confertus :: Brisbane Box     SOUTHERN   1835
  7           Arbutus 'Marina' :: Hybrid Strawberry Tree      TARAVAL   1763
  8       Platanus x hispanica :: Sycamore: London Plane      CENTRAL   1654
  9        Metrosideros excelsa :: New Zealand Xmas Tree     RICHMOND   1629
  10              Pittosporum undulatum :: Victorian Box     NORTHERN   1600
  11           Magnolia grandiflora :: Southern Magnolia      MISSION   1512
  12                                          Tree(s) ::    INGLESIDE   1510
  13                    Prunus cerasifera :: Cherry Plum      MISSION   1472
  14               Lophostemon confertus :: Brisbane Box      BAYVIEW   1451
  15               Lophostemon confertus :: Brisbane Box      MISSION   1404
  16                                          Tree(s) ::      TARAVAL   1387
  17      Platanus x hispanica :: Sycamore: London Plane      MISSION   1366
  18                    Prunus cerasifera :: Cherry Plum      TARAVAL   1348
  19   Ficus microcarpa nitida 'Green Gem' :: Indian ...     NORTHERN   1334
  20                    Prunus cerasifera :: Cherry Plum         PARK   1272
  21              Pittosporum undulatum :: Victorian Box         PARK   1229
  22               Tristaniopsis laurina :: Swamp Myrtle      TARAVAL   1216
  23              Pittosporum undulatum :: Victorian Box      MISSION   1210
  24       Metrosideros excelsa :: New Zealand Xmas Tree         PARK   1205
  25          Arbutus 'Marina' :: Hybrid Strawberry Tree    INGLESIDE   1195
  26       Metrosideros excelsa :: New Zealand Xmas Tree    INGLESIDE   1181
  27   Ficus microcarpa nitida 'Green Gem' :: Indian ...      CENTRAL   1092
  28           Magnolia grandiflora :: Southern Magnolia     NORTHERN    957
  29                    Prunus cerasifera :: Cherry Plum    INGLESIDE    948
  ..                                                 ...          ...    ...
  170                          Maytenus boaria :: Mayten      CENTRAL     90
  171              Tristaniopsis laurina :: Swamp Myrtle      CENTRAL     88
  172          Magnolia grandiflora :: Southern Magnolia   TENDERLOIN     87
  173              Prunus serrulata :: Ornamental Cherry     SOUTHERN     85
  174                Pyrus calleryana :: Ornamental Pear      CENTRAL     83
  175              Prunus serrulata :: Ornamental Cherry     RICHMOND     77
  176          Callistemon citrinus :: Lemon Bottlebrush     SOUTHERN     74
  177  Prunus serrulata 'Kwanzan' :: Kwanzan Flowerin...     SOUTHERN     66
  178          Callistemon citrinus :: Lemon Bottlebrush      CENTRAL     66
  179              Tristaniopsis laurina :: Swamp Myrtle   TENDERLOIN     62
  180         Arbutus 'Marina' :: Hybrid Strawberry Tree      CENTRAL     61
  181  Prunus serrulata 'Kwanzan' :: Kwanzan Flowerin...   TENDERLOIN     53
  182  Ficus microcarpa nitida 'Green Gem' :: Indian ...      TARAVAL     51
  183              Prunus serrulata :: Ornamental Cherry      CENTRAL     47
  184         Arbutus 'Marina' :: Hybrid Strawberry Tree   TENDERLOIN     46
  185            Corymbia ficifolia :: Red Flowering Gum   TENDERLOIN     42
  186  Prunus serrulata 'Kwanzan' :: Kwanzan Flowerin...      CENTRAL     39
  187             Acacia melanoxylon :: Blackwood Acacia     SOUTHERN     37
  188            Corymbia ficifolia :: Red Flowering Gum     SOUTHERN     35
  189                    Ulmus parvifolia :: Chinese Elm   

(Apologies for the ordering, as the graphing program ran into trouble due to the coloring)

The 20 most prominent tree species in SF are surprisingly well distributed.  One standout to me is the amount of Green Gem trees in the Tenderloin, as even though the district with the lowest tree count overall, it has a high amount of this tree type when compared to other tree types.  

## Where Are The Trees SF Is Known For?

# * More to come!*